In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
import numpy as np

plt.style.use('paper.mplstyle')

In [2]:
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

# table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue.fits' %root_dir)
table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_fixediso_10pc.fits' %root_dir)

table_muse = QTable.read('%s/data_misc/Nebulae_catalogue_v4/Nebulae_catalogue_v4.fits' %root_dir)

table = table[(table['flag_star_MUSE']!=1) & (table['flag_edge_MUSE']!=1) & (table["flag_edge_hst"]!=1)] # remove stars and edge objects
table_muse = table_muse[(table_muse['flag_star']!=1) & (table_muse['flag_edge']!=1)] # remove stars and edge objects

print(f'Table length post-masking (inc. HII regions only): {len(table)}')

Table length post-masking (inc. HII regions only): 5362


In [3]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]
columns = ['gal_name'] + [f'percentile_{p}' for p in percentiles]
data = []

for gal_name in np.unique(table['gal_name']):

    table_masked = table[table['gal_name'] == gal_name]
    table_muse_masked = table_muse[table_muse['gal_name'] == gal_name]

    gal_data = [gal_name.replace('NGC', 'NGC~').replace('IC', 'IC~')]
        
    p_lum = np.percentile(table_masked['HA6562_LUMINOSITY_HST'], percentiles).to('dex(erg/s)')
    p_rad = np.percentile(table_masked['radius_circ_pc'], percentiles)
    p_muse_rad = np.percentile(table_masked['region_circ_rad_pc_MUSE'], percentiles)
    p_muse_rad_ratio = np.percentile(table_masked['radius_circ_pc']/table_masked['region_circ_rad_pc_MUSE'], percentiles)

    for values in [p_lum, p_rad, p_muse_rad, p_muse_rad_ratio]:
        for value in values:
            gal_data.extend([value])
    
    data.append(gal_data)

In [4]:
# Calculate the percentiles for each "gal_name"
percentiles = [10, 50, 90]

gal_data = ['All']

p_lum = np.percentile(table['HA6562_LUMINOSITY_HST'], percentiles).to('dex(erg/s)')
p_rad = np.percentile(table['radius_circ_pc'], percentiles)
p_muse_rad = np.percentile(table['region_circ_rad_pc_MUSE'], percentiles)
p_muse_rad_ratio = np.percentile(table['radius_circ_pc']/table['region_circ_rad_pc_MUSE'], percentiles)

for values in [p_lum, p_rad, p_muse_rad, p_muse_rad_ratio]:
    for value in values:
        gal_data.extend([value])

data.append(gal_data)

In [5]:
# Create a new QTable with the calculated percentiles
names1 = ['gal_name']
names2 = [f'p{p}_lum' for p in percentiles]
names3 = [f'p{p}_rad' for p in percentiles]
names4 = [f'p{p}_muse_rad' for p in percentiles]
names5 = [f'p{p}_rad_ratio' for p in percentiles]

names = names1 + names2 + names3 + names4 + names5
new_table = QTable(rows=data, names=names)

new_table

gal_name,p10_lum,p50_lum,p90_lum,p10_rad,p50_rad,p90_rad,p10_muse_rad,p50_muse_rad,p90_muse_rad,p10_rad_ratio,p50_rad_ratio,p90_rad_ratio
,dex(erg / s),dex(erg / s),dex(erg / s),pc,pc,pc,pc,pc,pc,,,
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC~5332,36.79074373670279,37.40689038769775,38.06479586319056,9.324678899869532,19.381478628549935,36.98294045814851,40.16252807536435,59.147387412092584,80.26347079142718,0.24191326529168655,0.354568631978044,0.5029604750845589
NGC~0628,37.01379162128957,37.6205529630047,38.38866901411616,10.978251421890922,21.140019037979783,47.203093071862426,39.33604492144887,60.18380336382286,95.32625197813688,0.2306673184056076,0.35227071784161595,0.5246860403234874
NGC~1087,37.08663407974769,37.848690325364096,38.91303360034424,12.853970067078556,30.19412929692673,84.88392740400859,51.29710043866201,76.08589573311988,139.11050114185855,0.23399520283297334,0.41653098955918766,0.6933644949101667
NGC~1300,37.134826211863675,37.73400612378115,38.730913954003626,11.99992168578596,23.822720335544656,65.34498287765028,58.76645141744895,74.91282063054364,119.62597711786296,0.1824763230801414,0.3350174989459489,0.5691834328616229
NGC~1365,37.14426708443285,38.23177493802275,40.122424032665236,12.274819704647058,39.27484566722177,150.8439795006193,77.2008369956835,96.0541609931615,182.78131183177067,0.14427547715122663,0.417473234167666,0.9416500208442331
NGC~1385,37.28684800370148,38.18384332608336,39.31662267900049,14.694099228209405,43.4857389297616,117.91392107236496,48.03406284888858,81.85283819054975,149.09610457334495,0.27040401134398,0.5191791986013603,0.888124699084432
NGC~1433,36.95869117278865,37.671808080895865,38.56646381104584,8.846148770609842,21.366620695430957,60.04816651560787,58.54628242932918,72.06549195523239,119.53186368400395,0.13822692761612254,0.30146299764211115,0.5610659825518604
NGC~1512,36.87858203831007,37.68468582391812,38.9134793553805,10.243178511088516,24.402238975236262,100.22893336624648,79.12356420177082,84.63085805126394,176.19070507198376,0.12138049926644161,0.28415788951454907,0.5591257718460332


In [6]:
names = ['gal_name',
        'p10_lum',
        'p50_lum',
        'p90_lum',
        'p10_rad',
        'p50_rad',
        'p90_rad',
        'p10_muse_rad',
        'p50_muse_rad',
        'p90_muse_rad',
        'p10_rad_ratio',
        'p50_rad_ratio',
        'p90_rad_ratio']

formats={
        'gal_name': '%s',
        'p10_lum': '%0.1f',
        'p50_lum': '%0.1f',
        'p90_lum': '%0.1f',
        'p10_rad': '%0.1f',
        'p50_rad': '%0.1f',
        'p90_rad': '%0.1f', 
        'p10_muse_rad': '%0.1f',
        'p50_muse_rad': '%0.1f',
        'p90_muse_rad': '%0.1f',
        'p10_rad_ratio': '%0.2f',
        'p50_rad_ratio': '%0.2f',
        'p90_rad_ratio': '%0.2f'
        }

new_table.write('./tabs/lumrad_props.tex', formats=formats, names=names, overwrite=True)
# new_table.write('./tabs/lumrad_props.fits', overwrite=True)
# new_table.write('./tabs/lumrad_props.dat', format='ascii', overwrite=True)